In [4]:
from datasets import load_dataset

# Load only the test split from the dataset
test_data = load_dataset("cfilt/iitb-english-hindi", split="test")

# You now have the test data
print(test_data)


c:\Users\Deevia\miniconda3\envs\gpt\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Deevia\miniconda3\envs\gpt\Lib\site-packages\huggingface_hub\file_download.py:148: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Deevia\.cache\huggingface\hub\datasets--cfilt--iitb-english-hindi. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see act

Dataset({
    features: ['translation'],
    num_rows: 2507
})


In [6]:

import torch
#from datasets import load_dataset
from tokenizers import tokenizers
from tokenizers.models import WordLevel
from torch.utils.data import Dataset, DataLoader, random_split

from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
from pathlib import Path
import random
random.seed(42)
def get_config():
    return {
        "batch_size": 8,
        "num_epochs": 20,
        "lr": 10**-4,
        "seq_len": 350,
        "d_model": 512,
        "datasource": 'opus_books',
        "lang_src": "en",
        "lang_tgt": "it",
        "model_folder": "weights",
        "model_basename": "tmodel_",
        "preload": "latest",
        "tokenizer_file": "tokenizer_{0}.json",
        "experiment_name": "runs/tmodel"
    }

config = get_config()

def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]

def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer


# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, test_data, 'en')
tokenizer_tgt = get_or_build_tokenizer(config, test_data, 'hi')

In [7]:
tokenizer_src.get_vocab()


{'solids': 725,
 'War': 3350,
 'voices': 4354,
 'spotted': 1145,
 'week': 248,
 'Air': 2174,
 'Goon': 3098,
 'Google': 320,
 'facilities': 1277,
 'secretary': 1625,
 'channels': 1865,
 'Festival': 2233,
 'blonde': 1845,
 'drug': 622,
 'penis': 3997,
 'Fernando': 3073,
 'ceremonies': 3491,
 'Sea': 2321,
 'Commerce': 2218,
 'agree': 3391,
 'addition': 1810,
 'assign': 3429,
 'instruction': 2598,
 'controlled': 3544,
 'top': 430,
 'tiny': 4289,
 'disputes': 3624,
 'device': 1077,
 'run': 474,
 'Narayan': 2287,
 'prepaid': 4033,
 'Michael': 1427,
 'objective': 3962,
 'Conservator': 3032,
 'Silverado': 2329,
 'dynastic': 2515,
 'practice': 1591,
 'Indian': 743,
 'equipped': 1925,
 'date': 1255,
 'submitted': 2824,
 'ready': 954,
 'Neither': 3207,
 'eve': 3669,
 'popular': 4022,
 'unit': 2133,
 'cardinals': 450,
 'Post': 1771,
 'protein': 1597,
 'fight': 1281,
 'August': 487,
 'boys': 1850,
 'some': 107,
 'concerns': 1060,
 'Gender': 2241,
 'testes': 4280,
 '),': 484,
 'Usher': 3338,
 'techn

In [8]:
tokenizer_tgt.get_vocab()

{'वालों': 942,
 'वास्तविक': 943,
 'थीं': 618,
 '60': 861,
 'पायलट': 1624,
 'दृष्टि': 1349,
 'जा': 68,
 'पासपोर्ट': 1974,
 'कर्मचारी': 718,
 'निवासी': 912,
 'अंदरूनी': 2904,
 'लगने': 3918,
 'रहना': 2667,
 'आश्वस्त': 3008,
 'आमंत्रित': 1796,
 'बजाय': 586,
 'सहयोग': 850,
 'तथ्यों': 2430,
 'रंगकर्मी': 3863,
 'सेंट': 852,
 'बंगाल': 3658,
 'डेमोक्रेटिक': 3404,
 'जन्मदिन': 1575,
 'बीरा': 3697,
 'चैपल': 3304,
 'बिलकुल': 927,
 'सकता': 96,
 'कॉमेडी': 720,
 'चार्ली': 1871,
 'जाए': 354,
 'संदेशों': 2754,
 'SAP': 2898,
 'चलेगा': 3286,
 'मीडिया': 226,
 'प्राथमिकता': 2561,
 'मुक्त': 2042,
 'राशि': 1695,
 'पूर्वानुमान': 2538,
 'योजनाकार': 2659,
 'वार्ता': 2716,
 'निर्देशक': 3510,
 'बृजेंद्र': 3704,
 'जितना': 1579,
 'इस्लाम': 3033,
 'यूशर': 3858,
 'पादशाही': 3565,
 'कंपनियों': 401,
 'वेतन': 1442,
 'टैक्सी': 3374,
 'चयन': 2361,
 'मैंने': 3831,
 'कविता': 3142,
 'बोलते': 1668,
 'रूप': 41,
 'वर्षीय': 261,
 'वेल्लरटिगरा': 2734,
 '2000': 2876,
 'सहभागिता': 1461,
 'जन': 2383,
 'पहुंचे': 739,
 'सीरियल': 4093,


In [9]:
test_data[0]

{'translation': {'en': 'A black box in your car?',
  'hi': 'आपकी कार में ब्लैक बॉक्स?'}}

In [10]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class EnToHinDataset(Dataset):

    def __init__(self, ds, tk_src, tk_tgt, seq_len):
        super().__init__()
        self.seq_len = seq_len

        self.ds = ds
        self.tk_src = tk_src
        self.tk_tgt = tk_tgt

        # Special tokens
        self.sos = torch.tensor([tk_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos = torch.tensor([tk_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad = torch.tensor([tk_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        # Get the source and target text from the dataset
        dic = self.ds[idx]
        src_text = dic['translation']['en']
        tgt_text = dic['translation']['hi']

        # Tokenize the source and target text
        en = self.tk_src.encode(src_text).ids
        de = self.tk_tgt.encode(tgt_text).ids

        # Calculate the number of padding tokens needed
        enc_pad_len = self.seq_len - len(en) - 2  # for <sos> and <eos>
        dec_pad_len = self.seq_len - len(de) - 1  # only <sos> at the beginning

        # Check if the sentence is too long
        if enc_pad_len < 0 or dec_pad_len < 0:
            raise ValueError("Sentence is too long")

        # Create the encoder input by adding <sos>, <eos>, and padding
        en_inp = torch.cat([
            self.sos,
            torch.tensor(en, dtype=torch.int64),
            self.eos,
            torch.tensor([self.pad] * enc_pad_len, dtype=torch.int64)
        ])

        # Create the decoder input by adding <sos> and padding
        de_inp = torch.cat([
            self.sos,
            torch.tensor(de, dtype=torch.int64),
            torch.tensor([self.pad] * dec_pad_len, dtype=torch.int64)
        ])

        # Create the label by adding <eos> at the end and padding
        label = torch.cat([
            torch.tensor(de, dtype=torch.int64),
            self.eos,
            torch.tensor([self.pad] * dec_pad_len, dtype=torch.int64)
        ])

        # Return a dictionary containing the inputs and labels
        return {
            "encoder_input": en_inp,  # Encoder input
            "decoder_input": de_inp,  # Decoder input
            "label": label,  # Target labels
            "encoder_mask": (en_inp != self.pad).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (de_inp != self.pad).unsqueeze(0).int() & causal_mask(de_inp.size(0)), # (1, seq_len) & (1, seq_len, seq_len),
            "src_text": src_text,
            "tgt_text": tgt_text,
        }


def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

causal_mask(3)

tensor([[[ True, False, False],
         [ True,  True, False],
         [ True,  True,  True]]])

In [11]:
test_data

Dataset({
    features: ['translation'],
    num_rows: 2507
})

In [12]:
from torch.utils.data import Dataset, DataLoader, random_split
# Train and test splits
data = test_data
train_ds_size = int(0.9 * len(data))
val_ds_size = len(data) - train_ds_size
train_data, val_data = random_split(data, [train_ds_size, val_ds_size])

In [13]:
# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, test_data, 'en')
tokenizer_tgt = get_or_build_tokenizer(config, test_data, 'hi')

In [14]:
max_len_src = 0
max_len_tgt = 0
for item in test_data:
  src_ids = tokenizer_src.encode(item['translation']['en']).ids
  tgt_ids = tokenizer_tgt.encode(item['translation']['hi']).ids
  max_len_src = max(max_len_src, len(src_ids))
  max_len_tgt = max(max_len_tgt, len(tgt_ids))
print(f'Max length of source sentence: {max_len_src}')
print(f'Max length of target sentence: {max_len_tgt}')



Max length of source sentence: 90
Max length of target sentence: 92


In [15]:
seq_len = 100

t_ds = EnToHinDataset(train_data, tokenizer_src, tokenizer_tgt, 100)
v_ds = EnToHinDataset(val_data, tokenizer_src, tokenizer_tgt, 100)

In [16]:
from torch.utils.data import DataLoader

t_dl = DataLoader(t_ds, batch_size=16, shuffle=True)
v_dl = DataLoader(v_ds, batch_size=16, shuffle=True)

In [17]:
for i in t_ds:
  break
i

{'encoder_input': tensor([   2, 1001,    8,    4, 4365,    0, 1808,    6,  199,    4,  414,  418,
            7,  101,    4, 2574,    0, 2750,   34, 3679,  107, 2092,    9,    4,
          578, 2632,  498,   43, 4074,    6,   13,  616,  482,  707,   80,    4,
          625,    5,    3,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
            1,    1,    1,    1]),
 'decoder_input': tensor([   2, 1371, 3972,    9,    0,    4,   85,  947, 3411,   34, 2715,    4,
            0,  870,    6, 2159,   73, 1945,  870,    6, 1051,   14, 1352,  164,
           40,   33,    7,   36, 1166,  637,   58,  128,   47,    5,    1,    1,
            1,    1,    1,    1,    1,  

In [18]:
for i in t_dl:
  print(i['encoder_input'].shape)
  print(i['decoder_input'].shape)
  print(i['label'].shape)
  print(i['encoder_mask'].shape)
  print(i['decoder_mask'].shape)
  break

torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 100])
torch.Size([16, 1, 1, 100])
torch.Size([16, 1, 100, 100])


In [34]:
src_vocab_size = tokenizer_src.get_vocab_size()
tgt_vocab_size = tokenizer_tgt.get_vocab_size()
tgt_vocab_size

4180

# model building 

In [54]:
import torch
import torch.nn as nn
from torch.nn import functional as F

vocab_size = 100
batch_size = 16
block_size = 100  # seq_len
n_embd = 50
n_head = 2
n_layer = 2
dropout = 0.2
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_iters = 10000 # max no of steps for training 
eval_interval = 100 # after how many steps the evaluation will take place
eval_iters = 1000  # how many sample of batches will use for evaluation

lr = 3e-4




class DecoderHead(nn.Module):
    """ one head self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tri = torch.tril(torch.ones(block_size, block_size))
        self.dropout = nn.Dropout(dropout)

        self.register_buffer('tril', self.tri)


    def forward(self, x, y ,z, mask):

        mask = mask.squeeze(1)
        B, T, C = x.shape
        k = self.key(x) # (B, T, h)
        q = self.query(x) # (B, T, h)
        v = self.value(x) # (B, T, h)
        
      

        att = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,h)@(B,h,T) -> (B,T,T)
        att = att.masked_fill(mask == 0, float('-inf'))
        att = F.softmax(att, dim= -1) # (B,T,T)
        att = self.dropout(att)

        out = att @ v # (B,T,T)@(B,T,h) -> (B,T,h)
        return out

class EnocderHead(nn.Module):
    """ one head self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tri = torch.tril(torch.ones(block_size, block_size))
        self.dropout = nn.Dropout(dropout)

        self.register_buffer('tril', self.tri)


    def forward(self, x, mask=None):

        mask = mask.reshape(16,1, 100)
        B, T, C = x.shape
        k = self.key(x) # (B, T, h)
        q = self.query(x) # (B, T, h)
        v = self.value(x) # (B, T, h)

        att = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,h)@(B,h,T) -> (B,T,T)
        if mask is not None:
            att = att.masked_fill(mask == 0, float('-inf'))
        att = F.softmax(att, dim= -1) # (B,T,T)
        att = self.dropout(att)

        out = att @ v # (B,T,T)@(B,T,h) -> (B,T,h)

        return out

class EncoderMultiHeadAtt(nn.Module):

    def __init__(self,num_heads,  head_size):
        super().__init__()
        self.hd = nn.ModuleList([EnocderHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x, mask):
        out = torch.cat([h(x, mask) for h in self.hd], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
    
class DecoderMultiHeadAtt(nn.Module):

    def __init__(self,num_heads,  head_size):
        super().__init__()
        self.hd = nn.ModuleList([DecoderHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, y ,z, mask):
        out = torch.cat([h(x,  y ,z, mask) for h in self.hd], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
class EncoderBlock(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.mhead = EncoderMultiHeadAtt(n_head, head_size )
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, s_mask):

        # x = x + self.mhead(x)   #skip connections
        # x = x + self.ffwd(x)

        x = x + self.mhead(self.ln1(x), s_mask)
        x = x + self.ffwd(self.ln2(x))
        return x
class DecoderBlock(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.mhead = DecoderMultiHeadAtt(n_head, head_size)
        self.croshead = DecoderMultiHeadAtt(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, e_out, s_mask, t_mask):

        # x = x + self.mhead(x)   #skip connections
        # x = x + self.ffwd(x)
        x = self.ln1(x)
        x = x + self.mhead(x, e_out, e_out, s_mask)
        x = x + self.mhead(self.ln1(x), e_out, e_out, s_mask)
        x = x + self.ffwd(self.ln2(x))
        return x

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )


    def forward(self, x):
        return self.net(x)

class GPTFromScratch(nn.Module):

    def __init__(self):
        super().__init__()
        self.embd_table_e = nn.Embedding(src_vocab_size, n_embd) # (vocab_size,C)
        self.pos_table_e = nn.Embedding(block_size, n_embd) # (T,C)

        self.embd_table_d = nn.Embedding(tgt_vocab_size, n_embd) # (vocab_size,C)
        self.pos_table_d = nn.Embedding(block_size, n_embd) # (T,C)


        # self.encoderblock = nn.Sequential(
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 )
        #self.encoderblock = EncoderBlock(n_embd, n_head)

        self.encoderblock = nn.ModuleList([EncoderBlock(n_embd, n_head) for _ in range(4)])
                     
        self.decoderblock = nn.ModuleList([DecoderBlock(n_embd, n_head) for _ in range(4)])
        # nn.Sequential(
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 )

        #self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])


        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        #self.lm_head_e = nn.Linear(n_embd, src_vocab_size)
        self.lm_head_d = nn.Linear(n_embd, tgt_vocab_size)

        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def encoder(self, src_idx, mask):
        B, T = src_idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table_e(src_idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table_e(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        #x = self.encoderblock(x, mask)
        for block in self.encoderblock:
            x = block(x, mask)
        x = self.ln_f(x) # (B,T,C)
        return x
        #logits = self.lm_head_e(x) # (B,T,vocab_size)


    def decoder(self, tgt_idx,  e_out, s_mask, t_mask):
        B, T = tgt_idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table_d(tgt_idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table_d(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        #x = self.decoderblock(x, e_out, s_mask, t_mask)

        for block in self.decoderblock:
            x = block(x, e_out, s_mask, t_mask)
        x = self.ln_f(x) # (B,T,C)

        logits = self.lm_head_d(x) # (B,T,vocab_size)

        return logits

    def forward(self, idx, targets=None):
        B, T = idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table(idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        x = self.block(x)
        x = self.ln_f(x) # (B,T,C)

        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_token):
        # idx is (B.T) array
        for _ in range(max_new_token):
            #crop ids to only consider last block_size tokens
            idx_cond =  idx[:, -block_size:] #(B,T)
            # predictions
            logits, loss = self(idx_cond)  #(B,T,C)
            # take only last time step
            logits = logits[:, -1, :]  #(B,-1, C) -> (B, T+1th, C)
            probs = F.softmax(logits, dim = -1)  #(B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sample in the running sequence
            idx = torch.cat((idx,idx_next), dim = 1) #(B, T+1)

        return idx  #(B, T+1)







model = GPTFromScratch()
m = model.to(device)
# print the no of params in the model
total_params = sum(p.numel() for p in m.parameters())
print(f"The total no of params in the model is {total_params}")

# create torch optimiser:
optimizer = torch.optim.AdamW(model.parameters(), lr= lr)


# training loop:
for iter in range(max_iters):

    # # evaluation of Loss on train and val
    # if iter % eval_interval == 0 or iter == max_iters -1:
    #     losses = estimate_loss()
    #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data

    s_xb =  i['encoder_input']
    t_xb =    i['decoder_input']
    s_msk= i['encoder_mask']
    t_msk= i['decoder_mask']
    targets = i['label']


    en_out = model.encoder(s_xb,s_msk)

    logits = model.decoder(t_xb,  en_out, s_msk, t_msk)

    print(en_out.shape)
    print(logits.shape)


    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
        loss = loss_fn(logits, targets)
        #loss = F.cross_entropy(logits, targets)
    

    print('loss:', loss.item())
    #print(logits.shape)
    # set grad = zero
    break
    optimizer.zero_grad(set_to_none=True)

    # back propagation:
        # grad calculation and param update
    loss.backward()
    optimizer.step()

The total no of params in the model is 935630
torch.Size([16, 100, 50])
torch.Size([16, 100, 4180])
loss: 8.355685234069824


In [58]:
import torch
import torch.nn as nn
from torch.nn import functional as F

#vocab_size = 100
batch_size = 16
block_size = 100  # seq_len
n_embd = 50
n_head = 2
n_layer = 2
dropout = 0.2
torch.manual_seed(1337)
device = 'cuda' if torch.cuda.is_available() else 'cpu'
max_iters = 10000 # max no of steps for training 
eval_interval = 100 # after how many steps the evaluation will take place
eval_iters = 1000  # how many sample of batches will use for evaluation

lr = 3e-4




class DecoderHead(nn.Module):
    """ one head self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tri = torch.tril(torch.ones(block_size, block_size))
        self.dropout = nn.Dropout(dropout)

        self.register_buffer('tril', self.tri)


    def forward(self, x, y ,z, mask):

        mask = mask.squeeze(1)
        B, T, C = x.shape
        k = self.key(x) # (B, T, h)
        q = self.query(x) # (B, T, h)
        v = self.value(x) # (B, T, h)
        
      

        att = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,h)@(B,h,T) -> (B,T,T)
        att = att.masked_fill(mask == 0, float('-inf'))
        att = F.softmax(att, dim= -1) # (B,T,T)
        att = self.dropout(att)

        out = att @ v # (B,T,T)@(B,T,h) -> (B,T,h)
        return out

class EnocderHead(nn.Module):
    """ one head self-attention"""

    def __init__(self, head_size):
        super().__init__()
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.tri = torch.tril(torch.ones(block_size, block_size))
        self.dropout = nn.Dropout(dropout)

        self.register_buffer('tril', self.tri)


    def forward(self, x, mask=None):

        mask = mask.reshape(16,1, 100)
        B, T, C = x.shape
        k = self.key(x) # (B, T, h)
        q = self.query(x) # (B, T, h)
        v = self.value(x) # (B, T, h)

        att = q @ k.transpose(-2,-1) * k.shape[-1]**-0.5 # (B,T,h)@(B,h,T) -> (B,T,T)
        if mask is not None:
            att = att.masked_fill(mask == 0, float('-inf'))
        att = F.softmax(att, dim= -1) # (B,T,T)
        att = self.dropout(att)

        out = att @ v # (B,T,T)@(B,T,h) -> (B,T,h)

        return out

class EncoderMultiHeadAtt(nn.Module):

    def __init__(self,num_heads,  head_size):
        super().__init__()
        self.hd = nn.ModuleList([EnocderHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self,x, mask):
        out = torch.cat([h(x, mask) for h in self.hd], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
    
class DecoderMultiHeadAtt(nn.Module):

    def __init__(self,num_heads,  head_size):
        super().__init__()
        self.hd = nn.ModuleList([DecoderHead(head_size) for _ in range(num_heads)])
        self.proj = nn.Linear(head_size * num_heads, n_embd)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, y ,z, mask):
        out = torch.cat([h(x,  y ,z, mask) for h in self.hd], dim=-1)
        out = self.proj(out)
        out = self.dropout(out)
        return out
class EncoderBlock(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.mhead = EncoderMultiHeadAtt(n_head, head_size )
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, s_mask):

        # x = x + self.mhead(x)   #skip connections
        # x = x + self.ffwd(x)

        x = x + self.mhead(self.ln1(x), s_mask)
        x = x + self.ffwd(self.ln2(x))
        return x
class DecoderBlock(nn.Module):

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.mhead = DecoderMultiHeadAtt(n_head, head_size)
        self.croshead = DecoderMultiHeadAtt(n_head, head_size)
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x, e_out, s_mask, t_mask):

        # x = x + self.mhead(x)   #skip connections
        # x = x + self.ffwd(x)
        x = self.ln1(x)
        x = x + self.mhead(x, e_out, e_out, s_mask)
        x = x + self.mhead(self.ln1(x), e_out, e_out, s_mask)
        x = x + self.ffwd(self.ln2(x))
        return x

class FeedFoward(nn.Module):

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )


    def forward(self, x):
        return self.net(x)

class GPTFromScratch(nn.Module):

    def __init__(self):
        super().__init__()
        self.embd_table_e = nn.Embedding(src_vocab_size, n_embd) # (vocab_size,C)
        self.pos_table_e = nn.Embedding(block_size, n_embd) # (T,C)

        self.embd_table_d = nn.Embedding(tgt_vocab_size, n_embd) # (vocab_size,C)
        self.pos_table_d = nn.Embedding(block_size, n_embd) # (T,C)


        # self.encoderblock = nn.Sequential(
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 EncoderBlock(n_embd, n_head),
        #                 )
        #self.encoderblock = EncoderBlock(n_embd, n_head)

        self.encoderblock = nn.ModuleList([EncoderBlock(n_embd, n_head) for _ in range(4)])
                     
        self.decoderblock = nn.ModuleList([DecoderBlock(n_embd, n_head) for _ in range(4)])
        # nn.Sequential(
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 DecoderBlock(n_embd, n_head),
        #                 )

        #self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])


        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        #self.lm_head_e = nn.Linear(n_embd, src_vocab_size)
        self.lm_head_d = nn.Linear(n_embd, tgt_vocab_size)

        self.apply(self._init_weights)


    def _init_weights(self, module):
        if isinstance(module, nn.Linear):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)
            if module.bias is not None:
                torch.nn.init.zeros_(module.bias)
        elif isinstance(module, nn.Embedding):
            torch.nn.init.normal_(module.weight, mean=0.0, std=0.02)


    def encoder(self, src_idx, mask):
        B, T = src_idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table_e(src_idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table_e(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        #x = self.encoderblock(x, mask)
        for block in self.encoderblock:
            x = block(x, mask)
        x = self.ln_f(x) # (B,T,C)
        return x
        #logits = self.lm_head_e(x) # (B,T,vocab_size)


    def decoder(self, tgt_idx,  e_out, s_mask, t_mask):
        B, T = tgt_idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table_d(tgt_idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table_d(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        #x = self.decoderblock(x, e_out, s_mask, t_mask)

        for block in self.decoderblock:
            x = block(x, e_out, s_mask, t_mask)
        x = self.ln_f(x) # (B,T,C)

        logits = self.lm_head_d(x) # (B,T,vocab_size)

        return logits

    def forward(self, idx, targets=None):
        B, T = idx.shape  # (B -> batch, T -> block_size(seq_len))

        # both xb, yb shape is (B,T) tensor of ints
        tok_emb = self.embd_table(idx) #o/p -> (B,T,C)
        pos_emb = self.pos_table(torch.arange(T, device=device))
        x  = tok_emb + pos_emb # (B,T,C)-> (B,T,C)+ (C,T)
        #x = self.head(x)
        # x = self.mhead(x)
        # x = self.ffwd(x)
        x = self.block(x)
        x = self.ln_f(x) # (B,T,C)

        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_token):
        # idx is (B.T) array
        for _ in range(max_new_token):
            #crop ids to only consider last block_size tokens
            idx_cond =  idx[:, -block_size:] #(B,T)
            # predictions
            logits, loss = self(idx_cond)  #(B,T,C)
            # take only last time step
            logits = logits[:, -1, :]  #(B,-1, C) -> (B, T+1th, C)
            probs = F.softmax(logits, dim = -1)  #(B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) #(B,1)
            # append sample in the running sequence
            idx = torch.cat((idx,idx_next), dim = 1) #(B, T+1)

        return idx  #(B, T+1)







model = GPTFromScratch()
m = model.to(device)
# print the no of params in the model
total_params = sum(p.numel() for p in m.parameters())
print(f"The total no of params in the model is {total_params}")

# create torch optimiser:
optimizer = torch.optim.AdamW(model.parameters(), lr= lr)


# training loop:
for iter , batch in enumerate(t_dl):

    # # evaluation of Loss on train and val
    # if iter % eval_interval == 0 or iter == max_iters -1:
    #     losses = estimate_loss()
    #     print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data

    s_xb = batch['encoder_input'].to(device) # (b, seq_len)
    t_xb = batch['decoder_input'].to(device) # (B, seq_len)
    s_msk = batch['encoder_mask'].to(device) # (B, 1, 1, seq_len)
    t_msk = batch['decoder_mask'].to(device) # (B, 1, seq_len, seq_len)

    en_out = model.encoder(s_xb,s_msk)

    logits = model.decoder(t_xb,  en_out, s_msk, t_msk)

    # print(en_out.shape)
    # print(logits.shape)


    if targets is None:
        loss = None
    else:
        B, T, C = logits.shape
        logits = logits.view(B*T, C)
        targets = targets.view(B*T)
        loss_fn = nn.CrossEntropyLoss(ignore_index=tokenizer_src.token_to_id('[PAD]'), label_smoothing=0.1).to(device)
        loss = loss_fn(logits, targets)
        #loss = F.cross_entropy(logits, targets)
    

    print('loss:', loss.item())
    #print(logits.shape)
    # set grad = zero

    optimizer.zero_grad(set_to_none=True)

    # back propagation:
        # grad calculation and param update
    loss.backward()
    optimizer.step()

The total no of params in the model is 935630
loss: 8.351285934448242
loss: 8.336915969848633
loss: 8.32962417602539
loss: 8.318343162536621
loss: 8.294999122619629
loss: 8.288289070129395
loss: 8.269145011901855
loss: 8.269899368286133
loss: 8.242206573486328
loss: 8.217208862304688
loss: 8.209543228149414
loss: 8.192745208740234
loss: 8.175320625305176
loss: 8.147579193115234
loss: 8.133208274841309
loss: 8.099715232849121
loss: 8.075833320617676
loss: 8.072498321533203
loss: 8.03001880645752
loss: 8.023053169250488
loss: 7.9666056632995605
loss: 7.945073127746582
loss: 7.930259704589844
loss: 7.897216796875
loss: 7.854577541351318
loss: 7.835141181945801
loss: 7.800683498382568
loss: 7.77372407913208
loss: 7.741453170776367
loss: 7.721309185028076
loss: 7.689691066741943
loss: 7.662416458129883
loss: 7.637540340423584
loss: 7.611914157867432
loss: 7.584750175476074
loss: 7.555949687957764
loss: 7.531854152679443
loss: 7.50553560256958
loss: 7.4775166511535645
loss: 7.450978279113769

In [34]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset

class BilingualDataset(Dataset):

    def __init__(self, ds, tokenizer_src, tokenizer_tgt, src_lang, tgt_lang, seq_len):
        super().__init__()
        self.seq_len = seq_len

        self.ds = ds
        self.tokenizer_src = tokenizer_src
        self.tokenizer_tgt = tokenizer_tgt
        self.src_lang = src_lang
        self.tgt_lang = tgt_lang

        self.sos_token = torch.tensor([tokenizer_tgt.token_to_id("[SOS]")], dtype=torch.int64)
        self.eos_token = torch.tensor([tokenizer_tgt.token_to_id("[EOS]")], dtype=torch.int64)
        self.pad_token = torch.tensor([tokenizer_tgt.token_to_id("[PAD]")], dtype=torch.int64)

    def __len__(self):
        return len(self.ds)

    def __getitem__(self, idx):
        src_target_pair = self.ds[idx]
        src_text = src_target_pair['translation'][self.src_lang]
        tgt_text = src_target_pair['translation'][self.tgt_lang]

        # Transform the text into tokens
        enc_input_tokens = self.tokenizer_src.encode(src_text).ids
        dec_input_tokens = self.tokenizer_tgt.encode(tgt_text).ids

        # Add sos, eos and padding to each sentence
        enc_num_padding_tokens = self.seq_len - len(enc_input_tokens) - 2  # We will add <s> and </s>
        # We will only add <s>, and </s> only on the label
        dec_num_padding_tokens = self.seq_len - len(dec_input_tokens) - 1

        # Make sure the number of padding tokens is not negative. If it is, the sentence is too long
        if enc_num_padding_tokens < 0 or dec_num_padding_tokens < 0:
            raise ValueError("Sentence is too long")

        # Add <s> and </s> token
        encoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(enc_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * enc_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only <s> token
        decoder_input = torch.cat(
            [
                self.sos_token,
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Add only </s> token
        label = torch.cat(
            [
                torch.tensor(dec_input_tokens, dtype=torch.int64),
                self.eos_token,
                torch.tensor([self.pad_token] * dec_num_padding_tokens, dtype=torch.int64),
            ],
            dim=0,
        )

        # Double check the size of the tensors to make sure they are all seq_len long
        assert encoder_input.size(0) == self.seq_len
        assert decoder_input.size(0) == self.seq_len
        assert label.size(0) == self.seq_len

        return {
            "encoder_input": encoder_input,  # (seq_len)
            "decoder_input": decoder_input,  # (seq_len)
            "encoder_mask": (encoder_input != self.pad_token).unsqueeze(0).unsqueeze(0).int(), # (1, 1, seq_len)
            "decoder_mask": (decoder_input != self.pad_token).unsqueeze(0).int() & causal_mask(decoder_input.size(0)), # (1, seq_len) & (1, seq_len, seq_len),
            "label": label,  # (seq_len)
            "src_text": src_text,
            "tgt_text": tgt_text,
        }

def causal_mask(size):
    mask = torch.triu(torch.ones((1, size, size)), diagonal=1).type(torch.int)
    return mask == 0

In [35]:
from datasets import load_dataset
from tokenizers import Tokenizer
from tokenizers.models import WordLevel
from tokenizers.trainers import WordLevelTrainer
from tokenizers.pre_tokenizers import Whitespace
def get_all_sentences(ds, lang):
    for item in ds:
        yield item['translation'][lang]
def get_or_build_tokenizer(config, ds, lang):
    tokenizer_path = Path(config['tokenizer_file'].format(lang))
    if not Path.exists(tokenizer_path):
        # Most code taken from: https://huggingface.co/docs/tokenizers/quicktour
        tokenizer = Tokenizer(WordLevel(unk_token="[UNK]"))
        tokenizer.pre_tokenizer = Whitespace()
        trainer = WordLevelTrainer(special_tokens=["[UNK]", "[PAD]", "[SOS]", "[EOS]"], min_frequency=2)
        tokenizer.train_from_iterator(get_all_sentences(ds, lang), trainer=trainer)
        tokenizer.save(str(tokenizer_path))
    else:
        tokenizer = Tokenizer.from_file(str(tokenizer_path))
    return tokenizer

ds_raw = load_dataset(f"{config['datasource']}", f"{config['lang_src']}-{config['lang_tgt']}", split='train')

# Build tokenizers
tokenizer_src = get_or_build_tokenizer(config, ds_raw, config['lang_src'])
tokenizer_tgt = get_or_build_tokenizer(config, ds_raw, config['lang_tgt'])

# Keep 90% for training, 10% for validation
train_ds_size = int(0.9 * len(ds_raw))
val_ds_size = len(ds_raw) - train_ds_size
train_ds_raw, val_ds_raw = random_split(ds_raw, [train_ds_size, val_ds_size])

train_ds = BilingualDataset(train_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])
val_ds = BilingualDataset(val_ds_raw, tokenizer_src, tokenizer_tgt, config['lang_src'], config['lang_tgt'], config['seq_len'])

# Find the maximum length of each sentence in the source and target sentence
max_len_src = 0
max_len_tgt = 0

for item in ds_raw:
    src_ids = tokenizer_src.encode(item['translation'][config['lang_src']]).ids
    tgt_ids = tokenizer_tgt.encode(item['translation'][config['lang_tgt']]).ids
    max_len_src = max(max_len_src, len(src_ids))
    max_len_tgt = max(max_len_tgt, len(tgt_ids))

print(f'Max length of source sentence: {max_len_src}')
print(f'Max length of target sentence: {max_len_tgt}')


train_dataloader = DataLoader(train_ds, batch_size=config['batch_size'], shuffle=True)
val_dataloader = DataLoader(val_ds, batch_size=1, shuffle=True)

Max length of source sentence: 309
Max length of target sentence: 274


In [36]:
for i in train_dataloader:
  print(i)
  break

{'encoder_input': tensor([[    2,   196,   499,  ...,     1,     1,     1],
        [    2,    56,     0,  ...,     1,     1,     1],
        [    2,   503,    39,  ...,     1,     1,     1],
        ...,
        [    2,    51,    44,  ...,     1,     1,     1],
        [    2, 27130,   352,  ...,     1,     1,     1],
        [    2,   129,    44,  ...,     1,     1,     1]]), 'decoder_input': tensor([[    2, 12959,    14,  ...,     1,     1,     1],
        [    2,     0,     6,  ...,     1,     1,     1],
        [    2,   146,   216,  ...,     1,     1,     1],
        ...,
        [    2,    47,    44,  ...,     1,     1,     1],
        [    2,  2345,  5382,  ...,     1,     1,     1],
        [    2,   223,    12,  ...,     1,     1,     1]]), 'encoder_mask': tensor([[[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        ...,


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        [[[1, 1, 1,  ..., 0, 0, 0]]],


        

In [39]:
i['encoder_input'].shape

torch.Size([8, 350])

In [47]:
i['encoder_input'][0][:20] #sos eos

tensor([    2,   196,   499,    54,   522,     6,    25,     9,   872,    10,
           38, 22357,   988,  3561,     3,     1,     1,     1,     1,     1])

In [48]:
i['decoder_input'][0][:20] #sos

tensor([    2, 12959,    14,   486,    60,  1963,    13,   455,    16,    99,
          520,  4107,    30,     1,     1,     1,     1,     1,     1,     1])

In [49]:
i['label'][0][:20] #eos

tensor([12959,    14,   486,    60,  1963,    13,   455,    16,    99,   520,
         4107,    30,     3,     1,     1,     1,     1,     1,     1,     1])